In [1]:
import pandas as pd
import json 
import numpy as np
from tqdm import tqdm
import ast
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
from clayrs import content_analyzer as ca
from clayrs import recsys as rs
from clayrs import evaluation as eva

In [399]:
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
import math

In [5]:
from scipy.special import kl_div

# Necessary functions

In [6]:
def embeddings_to_df(e):
    e_list = e.tolist()
    df_e = pd.DataFrame(e_list)
    return df_e

In [7]:
def get_performances(train, test, k=10):
    em = eva.EvalModel(
        [train],
        [test],
        metric_list=[
            eva.PrecisionAtK(k, sys_average='macro'),
            eva.RecallAtK(k, sys_average='macro'),
            eva.FMeasureAtK(k, sys_average='macro'),
            eva.MRR(),
            eva.NDCGAtK(k),
            # eva.CatalogCoverage(catalog),
            # eva.GiniIndex()
        ],    
    )
    sys_result, users_result = em.fit()
    return sys_result, users_result

In [8]:
def triangular_matrix(m):
    m_tri = m.where(np.triu(np.ones(m.shape),k=1).astype(bool))
    return m_tri

In [9]:
def ILS(m):
    m_tri = triangular_matrix(m).stack().reset_index()
    m_tri.columns = ['i','j','similarity']
    ils = (m_tri['similarity'].sum())/len(m_tri)
    return ils

In [10]:
def compute_ils(recos, similarity_matrix, list_users):
    dict_ils = {}
    for u in tqdm(list_users):
        recos_user = recos[recos['user_id']==u]
        news_ids = recos_user['item_id'].tolist()
        news_ids.sort()
        sim_matrix_user = similarity_matrix[similarity_matrix.index.isin(news_ids)][news_ids]
        ils_user = ILS(sim_matrix_user)
        dict_key = {u:ils_user}
        dict_ils.update(dict_key)
    return dict_ils

In [11]:
def get_results_categories(initial_results):
    results_categories = initial_results.copy()
    results_categories = results_categories.rename(columns={'item_id':'NewsID'})
    results_categories = results_categories.merge(news[['NewsID','cluster_hdbscan','proba']], on='NewsID').rename(columns={'cluster_hdbscan':'category'})
    results_categories['proba'] = results_categories['proba'].replace(0, 1)
    return results_categories

In [421]:
test = get_results_categories(results_a_0_k20)

In [432]:
test

,user_id,NewsID,score,category,proba
0,504290,47925,0.902574,0,1.000000
1,58570,47925,0.712538,0,1.000000
2,482240,47925,0.717136,0,1.000000
3,41646,47925,0.511041,0,1.000000
4,222828,47925,0.811083,0,1.000000
...,...,...,...,...,...
236476,61908,114172,0.000000,6,1.000000
236477,696889,85813,0.000000,11,0.806009
236478,330007,58354,0.000000,6,1.000000
236479,596721,58476,0.000000,4,0.876063


In [12]:
def compute_dcg(gain_vector):
    dcg_value = 0
    for i in range(1, len(gain_vector)+1, 1):
        val = gain_vector[i-1]/(math.log2(1+i))
        dcg_value = dcg_value + val
    return dcg_value

In [536]:
def alpha_ndcg(recos, list_users, users_interest_df, alpha, k=10):
    dict_alpha_ndcg = {}
    #For each user
    for u in tqdm(list_users):
        #Get the recommendation list of a specific user
        recos_user = recos[recos['user_id']==u].reset_index(drop=True)
        if len(recos_user)>=k:
            recos_user = recos_user.iloc[:k,:]
        #Get the corresponding list of news categories 
        recos_categories = recos_user['category'].tolist()
        
        #Initialize the gain vector
        gain_vector = []
        #Only compute alpha ndcg when there is 10 recos or more

        for i in range(1,len(recos_user)+1, 1):
            #Get the category
            cat = recos_categories[i-1]
            #Get the interest of the user for the category
            user_interest_cat = users_interest.loc[u,cat]
            if i ==1:
                r = 0
            else:
                r = recos_categories[:i-1].count(cat)
            #Compute the gain for the k-th reco
            # value_gain_k = recos_user.loc[i-1,'proba'] *  user_interest_cat * (1-alpha)**(r)
            # value_gain_k =  user_interest_cat * (1-alpha)**(r)
            #Update the gain vector
            gain_vector.append(value_gain_k)
        #Get the optimal gain vector (sorted gain vector in descending order)
        optimal_gain_vector = sorted(gain_vector, reverse=True)
        #Compute Discounted Cumulative Gain
        dcg = compute_dcg(gain_vector)
        #Compute Ideal Discounted Cumulative Gain
        idcg = compute_dcg(optimal_gain_vector)
        #Compute alpha-ndcg
        alpha_ndcg = dcg/idcg
        
        dict_key = {u:alpha_ndcg}
        dict_alpha_ndcg.update(dict_key)
        # else:
        #     dict_key = {u:np.nan}
        #     dict_alpha_ndcg.update(dict_key)
    return dict_alpha_ndcg        

In [14]:
def ndcg_ia(recos, list_users, users_interest_df, k=10):
    dict_ndcg_ia = {}
    for u in tqdm(list_users):
        recos_user = recos[recos['user_id']==u].reset_index(drop=True)
        if len(recos_user)>=k:
            recos_user = recos_user.iloc[:k,:]
        recos_categories = recos_user['category'].tolist()
        user_interest = pd.DataFrame(users_interest_df.loc[u])
        categories_interest = user_interest[user_interest[u]!=0].index.tolist()
        # nb_categories_interest = len(categories_interest)        
        list_values = []
        for c in range(len(categories_interest)):
            cat = categories_interest[c]
            user_interest_cat = users_interest.loc[u,cat]
            gain_vector_c = [1 if i==cat else 0 for i in recos_categories]
            optimal_gain_vector_c = sorted(gain_vector_c, reverse=True)
            dcg_c = compute_dcg(gain_vector_c)
            idcg_c = compute_dcg(optimal_gain_vector_c)
            if idcg_c != 0:
                ndcg_c = user_interest_cat*(dcg_c/idcg_c)
            else:
                ndcg_c = 0
            list_values.append(ndcg_c)
        ndcg_ia = sum(list_values)
        dict_key = {u:ndcg_ia}
        dict_ndcg_ia.update(dict_key)
        # else:
        #     dict_key = {u:np.nan}
        #     dict_ndcg_ia.update(dict_key)
    return dict_ndcg_ia

In [445]:
def s_recall(recos, list_users, categories_list, k):
    dict_srecall = {}
    nb_categories = len(categories_list)
    for u in tqdm(list_users):
        recos_user = recos[recos['user_id']==u].reset_index(drop=True)
        recos_categories = recos_user['category'].unique().tolist()
        s_recall_user = len(set(recos_categories))/nb_categories
        dict_key = {u:s_recall_user}
        dict_srecall.update(dict_key)
    return dict_srecall

In [137]:
def calibration_kl(recos, list_users, users_interest_df, categories_list, alpha=0.01):
    dict_ckl = {}
    for u in tqdm(list_users):
        interest_user = users_interest_df.loc[u].values.tolist()
        recos_user = recos[recos['user_id']==u].reset_index(drop=True)
        distrib_categories = []
        for c in categories_list:
            prop_cat = len(recos_user[recos_user['category']==c])
            distrib_categories.append(prop_cat/len(recos_user))
        q_distrib = [(1-alpha)*x + alpha*y for x, y in zip(distrib_categories, interest_user)]
        c_kl_user = kl_div(interest_user, q_distrib).sum()
        dict_key = {u:c_kl_user}
        dict_ckl.update(dict_key)
    return dict_ckl

In [138]:
from gensim.matutils import hellinger

In [139]:
def calibration_hellinger(recos, list_users, users_interest_df, categories_list):
    dict_ch = {}
    for u in tqdm(list_users):
        interest_user = users_interest_df.loc[u].values.tolist()
        recos_user = recos[recos['user_id']==u].reset_index(drop=True)
        distrib_categories = []
        for c in categories_list:
            prop_cat = len(recos_user[recos_user['category']==c])
            distrib_categories.append(prop_cat/len(recos_user))
        c_h_user = hellinger(interest_user, distrib_categories).sum()
        dict_key = {u:c_h_user}
        dict_ch.update(dict_key)
    return dict_ch

In [234]:
def homogeneization(distrib, param=0.5):
    n = len(distrib)
    new_distrib = [((1-param)*p)+(param/n) for p in distrib]
    return new_distrib

In [253]:
def calibration_hellinger_target(lambda_values, list_users, users_interest_df):
    dict_ch = {}
    for u in tqdm(list_users):
        interest_user = users_interest_df.loc[u].values.tolist()
        #get optimal lambda
        optimal_lambda = lambda_values.loc[u]['optimal_lambda']
        new_proportion = homogeneization(interest_user, param = optimal_lambda)
        # recos_user = recos[recos['user_id']==u].reset_index(drop=True)
        # distrib_categories = []
        # for c in categories_list:
        #     prop_cat = len(recos_user[recos_user['category']==c])
        #     distrib_categories.append(prop_cat/len(recos_user))
        c_h_user = hellinger(interest_user, new_proportion).sum()
        dict_key = {u:c_h_user}
        dict_ch.update(dict_key)
    return dict_ch

In [ ]:
def calibration_hellinger_target(lambda_values, list_users, users_interest_df):
    dict_ch = {}
    for u in tqdm(list_users):
        interest_user = users_interest_df.loc[u].values.tolist()
        #get optimal lambda
        optimal_lambda = lambda_values.loc[u]['optimal_lambda']
        new_proportion = homogeneization(interest_user, param = optimal_lambda)
        # recos_user = recos[recos['user_id']==u].reset_index(drop=True)
        # distrib_categories = []
        # for c in categories_list:
        #     prop_cat = len(recos_user[recos_user['category']==c])
        #     distrib_categories.append(prop_cat/len(recos_user))
        c_h_user = hellinger(interest_user, new_proportion).sum()
        dict_key = {u:c_h_user}
        dict_ch.update(dict_key)
    return dict_ch

In [551]:
# def calibration_hellinger_recos_target(recos, lambda_values, list_users, users_interest_df):
#     dict_ch = {}
#     for u in tqdm(list_users):
#         interest_user = users_interest_df.loc[u].values.tolist()
#         #get optimal lambda
#         optimal_lambda = lambda_values.loc[u]['optimal_lambda']
#         new_proportion = homogeneization(interest_user, param = optimal_lambda)
#         recos_user = recos[recos['user_id']==u].reset_index(drop=True)
#         distrib_categories = []
#         for c in categories_list:
#             prop_cat = len(recos_user[recos_user['category']==c])
#             distrib_categories.append(prop_cat/len(recos_user))
#         c_h_user = hellinger(distrib_categories, new_proportion).sum()
#         dict_key = {u:c_h_user}
#         dict_ch.update(dict_key)
#     return dict_ch

In [249]:
def get_all_results(results, test_set, users_list, interest, similarity_matrix, categories_list, name_parameters, k=10):
    print('Pre-processing...')
    results_categories = get_results_categories(results)  
    results_ratings = ca.Ratings.from_dataframe(results)
    test_ratings = ca.Ratings.from_dataframe(test_set)
    print('OK!')
    #Accuracy
    print('Accuracy metrics...')
    sys_results, users_results = get_performances(results_ratings, test_ratings, k=k)
    print('OK!')
    #Instantiate the dataframe with global results
    eval_results_global = sys_results.reset_index().copy()
    eval_results_global = eval_results_global[eval_results_global['user_id']=='sys - fold1']
    eval_results_global['user_id']=[name_parameters]
    eval_results_global = eval_results_global.rename(columns={'user_id':'a_value'})
    eval_results_global = eval_results_global.set_index('a_value')
    eval_results_global = eval_results_global.rename(columns={'Precision@10 - macro':'Precision', 'Recall@10 - macro':'Recall', 'F1@10 - macro':'F1 score', 'NDCG@10':'NDCG'})

    #Instantiate the dataframe with individual results
    eval_results_indiv = users_results.copy()
    eval_results_indiv.index = eval_results_indiv.index.astype(int)
    eval_results_indiv = eval_results_indiv.rename(columns={'Precision@10 - macro':'Precision', 'Recall@10 - macro':'Recall', 'F1@10 - macro':'F1 score', 'NDCG@10':'NDCG'})

    print('ILS...')
    #ILS
    dict_ils = compute_ils(results, similarity_matrix, users_list)
    eval_results_global['ILS'] = np.mean(list(dict_ils.values()))
    eval_results_indiv['ILS'] = eval_results_indiv.index.map(dict_ils)
    print('OK!')    
    print('alpha NDCG')
    #alpha-NDCG
    dict_alpha_ndcg = alpha_ndcg(results_categories, users_list, interest, alpha=0.5, k=k)
    eval_results_global['alpha_ndcg'] = np.mean(list(dict_alpha_ndcg.values()))
    eval_results_indiv['alpha_ndcg'] = eval_results_indiv.index.map(dict_alpha_ndcg)
    print('OK!')
    print('NDCG IA')
    #NDCG-IA
    dict_ndcg_ia = ndcg_ia(results_categories, users_list, interest, k=k)
    eval_results_global['ndcg_ia'] = np.mean(list(dict_ndcg_ia.values()))
    eval_results_indiv['ndcg_ia'] = eval_results_indiv.index.map(dict_ndcg_ia)
    print('OK!')
    print('S-Recall')
    #S-Recall
    dict_srecall = s_recall(results_categories, users_list, categories_list, k=k)
    eval_results_global['s_recall'] = np.mean(list(dict_srecall.values()))
    eval_results_indiv['s_recall'] = eval_results_indiv.index.map(dict_srecall)
    print('OK!')
    print('Calibration KL')
    #C_KL
    dict_ckl = calibration_kl(results_categories, users_list, interest, categories_list, alpha=0.01)
    eval_results_global['c_kl'] = np.mean(list(dict_ckl.values()))
    eval_results_indiv['c_kl'] = eval_results_indiv.index.map(dict_ckl)
    print('OK!')
    print('Calibration Hellinger')
    #C_KL
    dict_ch = calibration_hellinger(results_categories, users_list, interest, categories_list)
    eval_results_global['c_hell'] = np.mean(list(dict_ch.values()))
    eval_results_indiv['c_hell'] = eval_results_indiv.index.map(dict_ch)
    print('OK!')

    eval_results_global.columns = ['Precision', 'Recall', 'F1-score', 'MRR', 'NDCG', 'ILS', 'alpha_ndcg', 'ndcg_ia', 's_recall', 'c_kl', 'c_hell']
    eval_results_global = eval_results_global.round(3)

    eval_results_indiv.columns = ['Precision', 'Recall', 'F1-score', 'NDCG', 'ILS', 'alpha_ndcg', 'ndcg_ia', 's_recall', 'c_kl', 'c_hell']
    eval_results_indiv = eval_results_indiv.round(3)
    
    return eval_results_global, eval_results_indiv

# Necessary data

In [158]:
results_baseline_k20 = pd.read_csv('../../baseline/report_baseline_10k_cv_k20/CentroidVector_1/rs_rank_split0.csv')
test_baseline_k20 = pd.read_csv('../../baseline/report_baseline_10k_cv_k20/HoldOutPartitioning_test_split0.csv')

In [172]:
results_a_0_k20 = pd.read_csv('re_ranking_results/re_ranking/base/div_a_0_k20.csv')
# results_a_0_k20 = results_a_0_k20[['user_id_init','item_id','score']].rename(columns={'user_id_init':'user_id'})
results_a_0_k20 = results_a_0_k20[['user_id','item_id','score']]

results_a_01_k20 = pd.read_csv('re_ranking_results/re_ranking/base/div_a_01_k20.csv')
# results_a_01_k20 = results_a_01_k20[['user_id_init','item_id','score']].rename(columns={'user_id_init':'user_id'})
results_a_01_k20 = results_a_01_k20[['user_id','item_id','score']]

results_a_02_k20 = pd.read_csv('re_ranking_results/re_ranking/base/div_a_02_k20.csv')
# results_a_02_k20 = results_a_02_k20[['user_id_init','item_id','score']].rename(columns={'user_id_init':'user_id'})
results_a_02_k20 = results_a_02_k20[['user_id','item_id','score']]

results_a_03_k20 = pd.read_csv('re_ranking_results/re_ranking/base/div_a_03_k20.csv')
# results_a_03_k20 = results_a_03_k20[['user_id_init','item_id','score']].rename(columns={'user_id_init':'user_id'})
results_a_03_k20 = results_a_03_k20[['user_id','item_id','score']]

results_a_04_k20 = pd.read_csv('re_ranking_results/re_ranking/base/div_a_04_k20.csv')
# results_a_04_k20 = results_a_04_k20[['user_id_init','item_id','score']].rename(columns={'user_id_init':'user_id'})
results_a_04_k20 = results_a_04_k20[['user_id','item_id','score']]

results_a_05_k20 = pd.read_csv('re_ranking_results/re_ranking/base/div_a_05_k20.csv')
# results_a_05_k20 = results_a_05_k20[['user_id_init','item_id','score']].rename(columns={'user_id_init':'user_id'})
results_a_05_k20 = results_a_05_k20[['user_id','item_id','score']]

results_a_06_k20 = pd.read_csv('re_ranking_results/re_ranking/base/div_a_06_k20.csv')
# results_a_06_k20 = results_a_06_k20[['user_id_init','item_id','score']].rename(columns={'user_id_init':'user_id'})
results_a_06_k20 = results_a_06_k20[['user_id','item_id','score']]

results_a_07_k20 = pd.read_csv('re_ranking_results/re_ranking/base/div_a_07_k20.csv')
# results_a_07_k20 = results_a_07_k20[['user_id_init','item_id','score']].rename(columns={'user_id_init':'user_id'})
results_a_07_k20 = results_a_07_k20[['user_id','item_id','score']]

results_a_08_k20 = pd.read_csv('re_ranking_results/re_ranking/base/div_a_08_k20.csv')
# results_a_08_k20 = results_a_08_k20[['user_id_init','item_id','score']].rename(columns={'user_id_init':'user_id'})
results_a_08_k20 = results_a_08_k20[['user_id','item_id','score']]

results_a_09_k20 = pd.read_csv('re_ranking_results/re_ranking/base/div_a_09_k20.csv')
# results_a_09_k20 = results_a_09_k20[['user_id_init','item_id','score']].rename(columns={'user_id_init':'user_id'})
results_a_09_k20 = results_a_09_k20[['user_id','item_id','score']]

results_a_1_k20 = pd.read_csv('re_ranking_results/re_ranking/base/div_a_1_k20.csv')
# results_a_1_k20 = results_a_1_k20[['user_id_init','item_id','score']].rename(columns={'user_id_init':'user_id'})
results_a_1_k20 = results_a_1_k20[['user_id','item_id','score']]

In [277]:
entropy_a0 = pd.read_csv('re_ranking_results/ADF/entropy/entropy_a_0_k20.csv', index_col=0)
entropy_a01 = pd.read_csv('re_ranking_results/ADF/entropy/entropy_a_01_k20.csv', index_col=0)
entropy_a02 = pd.read_csv('re_ranking_results/ADF/entropy/entropy_a_02_k20.csv', index_col=0)
entropy_a03 = pd.read_csv('re_ranking_results/ADF/entropy/entropy_a_03_k20.csv', index_col=0)
entropy_a04 = pd.read_csv('re_ranking_results/ADF/entropy/entropy_a_04_k20.csv', index_col=0)
entropy_a05 = pd.read_csv('re_ranking_results/ADF/entropy/entropy_a_05_k20.csv', index_col=0)
entropy_a06 = pd.read_csv('re_ranking_results/ADF/entropy/entropy_a_06_k20.csv', index_col=0)
entropy_a07 = pd.read_csv('re_ranking_results/ADF/entropy/entropy_a_07_k20.csv', index_col=0)
entropy_a08 = pd.read_csv('re_ranking_results/ADF/entropy/entropy_a_08_k20.csv', index_col=0)
entropy_a09 = pd.read_csv('re_ranking_results/ADF/entropy/entropy_a_09_k20.csv', index_col=0)
entropy_a1 = pd.read_csv('re_ranking_results/ADF/entropy/entropy_a_1_k20.csv', index_col=0)

In [174]:
test_set = pd.read_csv('../../baseline/report_baseline_10k_cv_complete/HoldOutPartitioning_test_split0.csv')

In [23]:
# test_set = pd.read_csv('../report_subprofiles_cv_10k/HoldOutPartitioning_test_split0.csv')
# test_set['user_id_init'] = test_set['user_id'].str.split('-').str[0]
# test_set = test_set[['user_id_init','item_id','score']].rename(columns={'user_id_init':'user_id'})

In [175]:
users_list = [int(i) for i in test_set['user_id'].unique().tolist()]

In [25]:
# News info
news = pd.read_csv('../../thematic_clustering/lda_128_large/news_thematic_clustering_large_final.csv', index_col=0)

In [26]:
news['cluster_hdbscan'] = news['cluster_hdbscan']+1

In [27]:
# Embeddings df
embeddings_lda_128 = pd.read_json('../../baseline/news_codified_lda_128/contents.json')
embeddings_lda_128['lda_128#0'] = embeddings_lda_128['lda_128#0'].apply(ast.literal_eval)
news_embeddings_lda = embeddings_to_df(embeddings_lda_128['lda_128#0'])
news_embeddings_lda.index = embeddings_lda_128['content_id']

In [28]:
users_interest = pd.read_csv('../categories_distribution_subprofiles_10k.csv', index_col=0)

In [29]:
users_interest.columns = users_interest.columns.astype(int)

In [30]:
similarity_matrix = pd.DataFrame(cosine_similarity(news_embeddings_lda))

In [31]:
similarity_matrix.index = news_embeddings_lda.index.tolist()
similarity_matrix.columns = news_embeddings_lda.index.tolist()

In [32]:
categories_list = users_interest.columns.tolist()

# BASELINE

In [141]:
eval_baseline_k20_global, eval_baseline_k20_ind = get_all_results(results_baseline_k20, test_baseline_k20, users_list, users_interest, similarity_matrix, categories_list, 'baseline_k20', k=20)

Pre-processing...


INFO - Performing evaluation on metrics chosen


OK!
Accuracy metrics...


Performing NDCG@20:  80%|████████  | 4/5 [02:16<00:35]             C:\Users\ctreuill.BIDOUILLE\AppData\Roaming\Python\Python39\site-packages\clayrs\evaluation\metrics\ranking_metrics.py:205: RuntimeWarning: invalid value encountered in double_scalars
  return actual / ideal
Performing NDCG@20:  100%|██████████| 5/5 [02:55<00:00]


OK!
ILS...


...

OK!
Calibration KL


100%|██████████| 10000/10000 [00:59<00:00, 167.93it/s]


OK!
Calibration Hellinger


100%|██████████| 10000/10000 [00:49<00:00, 204.05it/s]

OK!


In [539]:
eval_baseline_k20_ind.to_csv('results_perfs/baseline/indiv/ind_baseline.cvs')

# All results ADF

## Compute performances

In [450]:
eval_a_0_k20_global, eval_a_0_k20_ind = get_all_results(results_a_0_k20, test_set, users_list, users_interest, similarity_matrix, categories_list, 'a_0_k20', k=20)
eval_a_01_k20_global, eval_a_01_k20_ind = get_all_results(results_a_01_k20, test_set, users_list, users_interest, similarity_matrix, categories_list, 'a_01_k20', k=20)
eval_a_02_k20_global, eval_a_02_k20_ind = get_all_results(results_a_02_k20, test_set, users_list, users_interest, similarity_matrix, categories_list, 'a_02_k20', k=20)
eval_a_03_k20_global, eval_a_03_k20_ind = get_all_results(results_a_03_k20, test_set, users_list, users_interest, similarity_matrix, categories_list, 'a_03_k20', k=20)
eval_a_04_k20_global, eval_a_04_k20_ind = get_all_results(results_a_04_k20, test_set, users_list, users_interest, similarity_matrix, categories_list, 'a_04_k20', k=20)
eval_a_05_k20_global, eval_a_05_k20_ind = get_all_results(results_a_05_k20, test_set, users_list, users_interest, similarity_matrix, categories_list, 'a_05_k20', k=20)
eval_a_06_k20_global, eval_a_06_k20_ind = get_all_results(results_a_06_k20, test_set, users_list, users_interest, similarity_matrix, categories_list, 'a_06_k20', k=20)
eval_a_07_k20_global, eval_a_07_k20_ind = get_all_results(results_a_07_k20, test_set, users_list, users_interest, similarity_matrix, categories_list, 'a_07_k20', k=20)
eval_a_08_k20_global, eval_a_08_k20_ind = get_all_results(results_a_08_k20, test_set, users_list, users_interest, similarity_matrix, categories_list, 'a_08_k20', k=20)
eval_a_09_k20_global, eval_a_09_k20_ind = get_all_results(results_a_09_k20, test_set, users_list, users_interest, similarity_matrix, categories_list, 'a_09_k20', k=20)
eval_a_1_k20_global, eval_a_1_k20_ind = get_all_results(results_a_1_k20, test_set, users_list, users_interest, similarity_matrix, categories_list, 'a_1_k20', k=20)

Pre-processing...


INFO - Performing evaluation on metrics chosen


OK!
Accuracy metrics...


Performing NDCG@20:  80%|████████  | 4/5 [01:23<00:20]             C:\Users\ctreuill.BIDOUILLE\AppData\Roaming\Python\Python39\site-packages\clayrs\evaluation\metrics\ranking_metrics.py:205: RuntimeWarning: invalid value encountered in double_scalars
  return actual / ideal
Performing NDCG@20:  100%|██████████| 5/5 [01:43<00:00]


OK!
ILS...


...

OK!
Calibration KL


100%|██████████| 10000/10000 [00:51<00:00, 192.83it/s]


OK!
Calibration Hellinger


100%|██████████| 10000/10000 [00:52<00:00, 190.90it/s]

OK!


In [451]:
c_hellinger_a0 = calibration_hellinger_target(entropy_a0, users_list, users_interest)
c_hellinger_a01 = calibration_hellinger_target(entropy_a01, users_list, users_interest)
c_hellinger_a02 = calibration_hellinger_target(entropy_a02, users_list, users_interest)
c_hellinger_a03 = calibration_hellinger_target(entropy_a03, users_list, users_interest)
c_hellinger_a04 = calibration_hellinger_target(entropy_a04, users_list, users_interest)
c_hellinger_a05 = calibration_hellinger_target(entropy_a05, users_list, users_interest)
c_hellinger_a06 = calibration_hellinger_target(entropy_a06, users_list, users_interest)
c_hellinger_a07 = calibration_hellinger_target(entropy_a07, users_list, users_interest)
c_hellinger_a08 = calibration_hellinger_target(entropy_a08, users_list, users_interest)
c_hellinger_a09 = calibration_hellinger_target(entropy_a09, users_list, users_interest)
c_hellinger_a1 = calibration_hellinger_target(entropy_a1, users_list, users_interest)

100%|██████████| 10000/10000 [00:02<00:00, 3793.15it/s]


In [552]:
# c_hellinger_a0_recos_target = calibration_hellinger_recos_target(get_results_categories(results_a_0_k20), entropy_a0, users_list, users_interest)
# c_hellinger_a01_recos_target = calibration_hellinger_recos_target(get_results_categories(results_a_01_k20), entropy_a01, users_list, users_interest)
# c_hellinger_a02_recos_target = calibration_hellinger_recos_target(get_results_categories(results_a_02_k20), entropy_a02, users_list, users_interest)
# c_hellinger_a03_recos_target = calibration_hellinger_recos_target(get_results_categories(results_a_03_k20), entropy_a03, users_list, users_interest)
# c_hellinger_a04_recos_target = calibration_hellinger_recos_target(get_results_categories(results_a_04_k20), entropy_a04, users_list, users_interest)
# c_hellinger_a05_recos_target = calibration_hellinger_recos_target(get_results_categories(results_a_05_k20), entropy_a05, users_list, users_interest)
# c_hellinger_a06_recos_target = calibration_hellinger_recos_target(get_results_categories(results_a_06_k20), entropy_a06, users_list, users_interest)
# c_hellinger_a07_recos_target = calibration_hellinger_recos_target(get_results_categories(results_a_07_k20), entropy_a07, users_list, users_interest)
# c_hellinger_a08_recos_target = calibration_hellinger_recos_target(get_results_categories(results_a_08_k20), entropy_a08, users_list, users_interest)
# c_hellinger_a09_recos_target = calibration_hellinger_recos_target(get_results_categories(results_a_09_k20), entropy_a09, users_list, users_interest)
# c_hellinger_a1_recos_target = calibration_hellinger_recos_target(get_results_categories(results_a_1_k20), entropy_a1, users_list, users_interest)

In [546]:
chl_a0 = np.mean(list(c_hellinger_a0_recos_target.values())).round(3)
chl_a01 = np.mean(list(c_hellinger_a01_recos_target.values())).round(3)
chl_a02 = np.mean(list(c_hellinger_a02_recos_target.values())).round(3)
chl_a03 = np.mean(list(c_hellinger_a03_recos_target.values())).round(3)
chl_a04 = np.mean(list(c_hellinger_a04_recos_target.values())).round(3)
chl_a05 = np.mean(list(c_hellinger_a05_recos_target.values())).round(3)
chl_a06 = np.mean(list(c_hellinger_a06_recos_target.values())).round(3)
chl_a07 = np.mean(list(c_hellinger_a07_recos_target.values())).round(3)
chl_a08 = np.mean(list(c_hellinger_a08_recos_target.values())).round(3)
chl_a09 = np.mean(list(c_hellinger_a09_recos_target.values())).round(3)
chl_a1 = np.mean(list(c_hellinger_a1_recos_target.values())).round(3)

In [554]:
# chl = [chl_a0, chl_a01, chl_a02, chl_a03, chl_a04, chl_a05, chl_a06, chl_a07, chl_a08, chl_a09, chl_a1]

In [553]:
chl = [0.141, 0.038, 0.030, 0.016, 0.055, 0.089, 0.110, 0.072, 0.041, 0.003, 0.011]

In [454]:
complete_results = pd.concat([eval_a_0_k20_global, eval_a_01_k20_global, eval_a_02_k20_global, eval_a_03_k20_global, eval_a_04_k20_global, eval_a_05_k20_global, eval_a_06_k20_global, eval_a_07_k20_global, eval_a_08_k20_global, eval_a_09_k20_global, eval_a_1_k20_global])

In [555]:
complete_results['c_hell_final'] = chl

In [559]:
complete_results = complete_results.iloc[::-1] 

In [560]:
complete_results.index = np.arange(0, 1.1, 0.1)

In [561]:
complete_results

,Precision,Recall,F1-score,MRR,NDCG,ILS,alpha_ndcg,ndcg_ia,s_recall,c_kl,c_hell,c_hell_target,c_hell_final
0.0,0.227,0.632,0.281,0.424,0.564,0.438,0.875,0.561,0.525,0.138,0.141,0.000,0.011
0.1,0.225,0.629,0.279,0.424,0.565,0.430,0.872,0.561,0.552,0.111,0.130,0.092,0.003
0.2,0.225,0.628,0.278,0.423,0.565,0.428,0.870,0.560,0.559,0.106,0.128,0.098,0.041
0.3,0.222,0.622,0.275,0.423,0.566,0.419,0.865,0.559,0.588,0.084,0.121,0.137,0.072
0.4,0.220,0.616,0.272,0.422,0.567,0.407,0.866,0.558,0.622,0.066,0.117,0.172,0.110
0.5,0.217,0.603,0.268,0.421,0.569,0.398,0.861,0.556,0.646,0.061,0.121,0.210,0.089
0.6,0.215,0.584,0.264,0.420,0.571,0.393,0.857,0.553,0.661,0.073,0.136,0.246,0.055
0.7,0.176,0.545,0.223,0.405,0.564,0.298,0.800,0.515,1.000,0.314,0.359,0.287,0.016
0.8,0.171,0.524,0.216,0.403,0.570,0.291,0.785,0.510,1.000,0.341,0.372,0.331,0.030
0.9,0.162,0.478,0.202,0.398,0.577,0.276,0.768,0.501,1.000,0.393,0.394,0.391,0.038


In [562]:
complete_results.to_csv('results_perfs/ADF/global_reranking_nosubprofiles.csv')

In [232]:
# a = 0
# for i in [eval_a_0_k20_ind, eval_a_01_k20_ind, eval_a_02_k20_ind, eval_a_03_k20_ind, eval_a_04_k20_ind, eval_a_05_k20_ind, eval_a_06_k20_ind, eval_a_07_k20_ind, eval_a_08_k20_ind, eval_a_09_k20_ind, eval_a_1_k20_ind]:
#     i = i.round(3)
#     path = 'results_perfs/ind_a_0'+str(a)+'.csv'
#     i.to_csv(path)
#     a = a+1

# All results - not personalized diversity

## Compute performances

In [301]:
test_df = pd.read_csv('re_ranking_results/ADF/re_ranking/base/div_a_0_k20.csv')

In [303]:
results_glob_a06 = pd.read_csv('re_ranking_results/global/re_ranking/div_global_0.6_k20.csv')
results_glob_a06 = results_glob_a06[['user_id','item_id','score']]

results_glob_a07 = pd.read_csv('re_ranking_results/global/re_ranking/div_global_0.7_k20.csv')
results_glob_a07 = results_glob_a07[['user_id','item_id','score']]

results_glob_a08 = pd.read_csv('re_ranking_results/global/re_ranking/div_global_0.8_k20.csv')
results_glob_a08 = results_glob_a08[['user_id','item_id','score']]

results_glob_a09 = pd.read_csv('re_ranking_results/global/re_ranking/div_global_0.9_k20.csv')
results_glob_a09 = results_glob_a09[['user_id','item_id','score']]

results_glob_a1 = pd.read_csv('re_ranking_results/global/re_ranking/div_global_1_k20.csv')
results_glob_a1 = results_glob_a1[['user_id','item_id','score']]

In [304]:
eval_a_06_k20_nopers, eval_a_06_k20_ind_nopers = get_all_results(results_glob_a06, test_set, users_list, users_interest, similarity_matrix, categories_list, 'a_06_k20_no_pers', k=20)
eval_a_07_k20_nopers, eval_a_07_k20_ind_nopers = get_all_results(results_glob_a07, test_set, users_list, users_interest, similarity_matrix, categories_list, 'a_07_k20_no_pers', k=20)
eval_a_08_k20_nopers, eval_a_08_k20_ind_nopers = get_all_results(results_glob_a08, test_set, users_list, users_interest, similarity_matrix, categories_list, 'a_08_k20_no_pers', k=20)
eval_a_09_k20_nopers, eval_a_09_k20_ind_nopers = get_all_results(results_glob_a09, test_set, users_list, users_interest, similarity_matrix, categories_list, 'a_09_k20_no_pers', k=20)
eval_a_1_k20_nopers, eval_a_1_k20_ind_nopers = get_all_results(results_glob_a1, test_set, users_list, users_interest, similarity_matrix, categories_list, 'a_1_k20_no_pers', k=20)

Pre-processing...


INFO - Performing evaluation on metrics chosen


OK!
Accuracy metrics...


Performing NDCG@20:  80%|████████  | 4/5 [01:33<00:23]             C:\Users\ctreuill.BIDOUILLE\AppData\Roaming\Python\Python39\site-packages\clayrs\evaluation\metrics\ranking_metrics.py:205: RuntimeWarning: invalid value encountered in double_scalars
  return actual / ideal
Performing NDCG@20:  100%|██████████| 5/5 [01:55<00:00]


OK!
ILS...


...

OK!
Calibration KL


100%|██████████| 10000/10000 [00:59<00:00, 169.09it/s]


OK!
Calibration Hellinger


100%|██████████| 10000/10000 [01:01<00:00, 163.31it/s]


OK!


In [305]:
complete_results_no_pers = pd.concat([eval_a_06_k20_nopers, eval_a_07_k20_nopers, eval_a_08_k20_nopers, eval_a_09_k20_nopers, eval_a_1_k20_nopers])

In [306]:
complete_results_no_pers

,Precision,Recall,F1-score,MRR,NDCG,ILS,alpha_ndcg,ndcg_ia,s_recall,c_kl,c_hell
a_value,,,,,,,,,,,
a_06_k20_no_pers,0.226,0.630,0.280,0.424,0.564,0.433,0.873,0.560,0.528,0.131,0.139
a_07_k20_no_pers,0.223,0.623,0.276,0.422,0.564,0.423,0.874,0.559,0.558,0.114,0.134
a_08_k20_no_pers,0.211,0.598,0.262,0.418,0.565,0.385,0.858,0.550,0.660,0.115,0.157
a_09_k20_no_pers,0.175,0.534,0.220,0.405,0.567,0.298,0.809,0.519,0.930,0.316,0.349
a_1_k20_no_pers,0.123,0.373,0.153,0.374,0.576,0.206,0.676,0.438,0.945,0.820,0.522


In [307]:
complete_results_no_pers.to_csv('results_perfs/no_pers/complete_results.csv')

In [351]:
eval_a_06_k20_ind_nopers.to_csv('results_perfs/no_pers/indiv/ind_h06.csv')
eval_a_07_k20_ind_nopers.to_csv('results_perfs/no_pers/indiv/ind_h07.csv')
eval_a_08_k20_ind_nopers.to_csv('results_perfs/no_pers/indiv/ind_h08.csv')
eval_a_08_k20_ind_nopers.to_csv('results_perfs/no_pers/indiv/ind_h09.csv')
eval_a_1_k20_ind_nopers.to_csv('results_perfs/no_pers/indiv/ind_h1.csv')


# All results - greedy

## Compute performances

In [381]:
results_greedy_0 = pd.read_csv('re_ranking_results/greedy/greedy_lambda_0.csv')

results_greedy_01 = pd.read_csv('re_ranking_results/greedy/greedy_lambda_01.csv')

results_greedy_02 = pd.read_csv('re_ranking_results/greedy/greedy_lambda_02.csv')

results_greedy_03 = pd.read_csv('re_ranking_results/greedy/greedy_lambda_03.csv')

results_greedy_04 = pd.read_csv('re_ranking_results/greedy/greedy_lambda_04.csv')

results_greedy_05 = pd.read_csv('re_ranking_results/greedy/greedy_lambda_05.csv')

results_greedy_06 = pd.read_csv('re_ranking_results/greedy/greedy_lambda_06.csv')

results_greedy_07 = pd.read_csv('re_ranking_results/greedy/greedy_lambda_07.csv')

results_greedy_08 = pd.read_csv('re_ranking_results/greedy/greedy_lambda_08.csv')

results_greedy_09 = pd.read_csv('re_ranking_results/greedy/greedy_lambda_09.csv')

results_greedy_1 = pd.read_csv('re_ranking_results/greedy/greedy_lambda_1.csv')

In [394]:
eval_greedy_0, eval_greedy_0_ind = get_all_results(results_greedy_0, test_set, users_list, users_interest, similarity_matrix, categories_list, 'greedy_0', k=20)
eval_greedy_01, eval_greedy_01_ind = get_all_results(results_greedy_01, test_set, users_list, users_interest, similarity_matrix, categories_list, 'greedy_01', k=20)
eval_greedy_02, eval_greedy_02_ind = get_all_results(results_greedy_02, test_set, users_list, users_interest, similarity_matrix, categories_list, 'greedy_02', k=20)
eval_greedy_03, eval_greedy_03_ind = get_all_results(results_greedy_03, test_set, users_list, users_interest, similarity_matrix, categories_list, 'greedy_03', k=20)
eval_greedy_04, eval_greedy_04_ind = get_all_results(results_greedy_04, test_set, users_list, users_interest, similarity_matrix, categories_list, 'greedy_04', k=20)
eval_greedy_05, eval_greedy_05_ind = get_all_results(results_greedy_05, test_set, users_list, users_interest, similarity_matrix, categories_list, 'greedy_05', k=20)
eval_greedy_06, eval_greedy_06_ind = get_all_results(results_greedy_06, test_set, users_list, users_interest, similarity_matrix, categories_list, 'greedy_06', k=20)
eval_greedy_07, eval_greedy_07_ind = get_all_results(results_greedy_07, test_set, users_list, users_interest, similarity_matrix, categories_list, 'greedy_07', k=20)
eval_greedy_08, eval_greedy_08_ind = get_all_results(results_greedy_08, test_set, users_list, users_interest, similarity_matrix, categories_list, 'greedy_08', k=20)
eval_greedy_09, eval_greedy_09_ind = get_all_results(results_greedy_09, test_set, users_list, users_interest, similarity_matrix, categories_list, 'greedy_09', k=20)
eval_greedy_1, eval_greedy_1_ind = get_all_results(results_greedy_1, test_set, users_list, users_interest, similarity_matrix, categories_list, 'greedy_1', k=20)

Pre-processing...


INFO - Performing evaluation on metrics chosen


OK!
Accuracy metrics...


Performing NDCG@20:  80%|████████  | 4/5 [01:30<00:21]             C:\Users\ctreuill.BIDOUILLE\AppData\Roaming\Python\Python39\site-packages\clayrs\evaluation\metrics\ranking_metrics.py:205: RuntimeWarning: invalid value encountered in double_scalars
  return actual / ideal
Performing NDCG@20:  100%|██████████| 5/5 [01:50<00:00]


OK!
ILS...


...

OK!
Calibration KL


100%|██████████| 10000/10000 [01:11<00:00, 139.54it/s]


OK!
Calibration Hellinger


100%|██████████| 10000/10000 [01:13<00:00, 136.25it/s]


OK!


In [395]:
complete_results_greedy = pd.concat([eval_greedy_0, eval_greedy_01, eval_greedy_02, eval_greedy_03, eval_greedy_04, eval_greedy_05, eval_greedy_06, eval_greedy_07, eval_greedy_08, eval_greedy_09, eval_greedy_1])

In [396]:
complete_results_greedy

,Precision,Recall,F1-score,MRR,NDCG,ILS,alpha_ndcg,ndcg_ia,s_recall,c_kl,c_hell
a_value,,,,,,,,,,,
greedy_0,0.224,0.670,0.282,0.425,0.563,0.550,0.890,0.518,0.383,0.797,0.364
greedy_01,0.224,0.670,0.282,0.425,0.563,0.550,0.890,0.518,0.383,0.796,0.363
greedy_02,0.224,0.670,0.282,0.426,0.564,0.549,0.890,0.519,0.383,0.793,0.363
greedy_03,0.225,0.671,0.282,0.427,0.564,0.548,0.890,0.519,0.384,0.789,0.362
greedy_04,0.225,0.672,0.282,0.430,0.565,0.545,0.890,0.520,0.386,0.775,0.359
greedy_05,0.224,0.671,0.282,0.420,0.557,0.528,0.886,0.522,0.395,0.741,0.353
greedy_06,0.223,0.669,0.281,0.415,0.553,0.519,0.887,0.522,0.399,0.736,0.352
greedy_07,0.223,0.669,0.280,0.411,0.550,0.508,0.886,0.524,0.406,0.714,0.348
greedy_08,0.222,0.667,0.279,0.401,0.541,0.475,0.875,0.523,0.427,0.668,0.339


In [398]:
complete_results_greedy.to_csv('results_perfs/greedy/complete_greedy.csv')

In [538]:
a = 0
for i in [eval_greedy_0_ind, eval_greedy_01_ind, eval_greedy_02_ind, eval_greedy_03_ind, eval_greedy_04_ind, eval_greedy_05_ind, eval_greedy_06_ind, eval_greedy_07_ind, eval_greedy_08_ind, eval_greedy_09_ind, eval_greedy_1_ind]:
    i = i.round(3)
    path = 'results_perfs/greedy/indiv/ind_a_0'+str(a)+'.csv'
    i.to_csv(path)
    a = a+1

# Comparison distributions

In [515]:
def compare_distrib(recos, users_list, users_interest_df):
    recos = get_results_categories(recos)
    dict_ch_smooth = {}
    for u in tqdm(users_list):
        best_ch = 10
        recos_user = recos[recos['user_id']==u].reset_index(drop=True)
        interest_user = users_interest_df.loc[u].values.tolist()
        distrib_cat_recos = []
        for c in categories_list:
            prop_cat = len(recos_user[recos_user['category']==c])
            distrib_cat_recos.append(prop_cat/len(recos_user))
        for l in np.arange(0, 1.1, 0.1):
            new_distrib = homogeneization(interest_user, param=round(l,1))
            c_h = hellinger(distrib_cat_recos,new_distrib)
            if c_h < best_ch:
                best_ch = c_h
                optimal_lambda = l
        dict_key = {u:best_ch}
        dict_ch_smooth.update(dict_key)
    return dict_ch_smooth

In [516]:
ch_baseline = compare_distrib(results_baseline_k20, users_list, users_interest)

ch_greedy0 = compare_distrib(results_greedy_0, users_list, users_interest)
ch_greedy01 = compare_distrib(results_greedy_01, users_list, users_interest)
ch_greedy02 = compare_distrib(results_greedy_02, users_list, users_interest)
ch_greedy03 = compare_distrib(results_greedy_03, users_list, users_interest)
ch_greedy04 = compare_distrib(results_greedy_04, users_list, users_interest)
ch_greedy05 = compare_distrib(results_greedy_05, users_list, users_interest)
ch_greedy06 = compare_distrib(results_greedy_06, users_list, users_interest)
ch_greedy07 = compare_distrib(results_greedy_07, users_list, users_interest)
ch_greedy08 = compare_distrib(results_greedy_08, users_list, users_interest)
ch_greedy09 = compare_distrib(results_greedy_09, users_list, users_interest)
ch_greedy1 = compare_distrib(results_greedy_1, users_list, users_interest)

  0%|          | 0/10000 [00:00<?, ?it/s]

100%|██████████| 10000/10000 [01:03<00:00, 157.47it/s]


In [517]:
chl_greedy0 = np.mean(list(ch_greedy0.values())).round(3)
chl_greedy01 = np.mean(list(ch_greedy01.values())).round(3)
chl_greedy02 = np.mean(list(ch_greedy02.values())).round(3)
chl_greedy03 = np.mean(list(ch_greedy03.values())).round(3)
chl_greedy04 = np.mean(list(ch_greedy04.values())).round(3)
chl_greedy05 = np.mean(list(ch_greedy05.values())).round(3)
chl_greedy06 = np.mean(list(ch_greedy06.values())).round(3)
chl_greedy07 = np.mean(list(ch_greedy07.values())).round(3)
chl_greedy08 = np.mean(list(ch_greedy08.values())).round(3)
chl_greedy09 = np.mean(list(ch_greedy09.values())).round(3)
chl_greedy1 = np.mean(list(ch_greedy1.values())).round(3)

chl_baseline = np.mean(list(ch_baseline.values())).round(3)


In [518]:
chl_greedy_complete = [chl_greedy0, chl_greedy01, chl_greedy02, chl_greedy03, chl_greedy04, chl_greedy05, chl_greedy06, chl_greedy07, chl_greedy08, chl_greedy09, chl_greedy1]

In [519]:
chl_greedy_complete

[0.36, 0.36, 0.359, 0.359, 0.356, 0.349, 0.348, 0.344, 0.335, 0.316, 0.37]

In [520]:
complete_results_greedy['c_hell_target'] = chl_greedy_complete

In [521]:
complete_results_greedy

,Precision,Recall,F1-score,MRR,NDCG,ILS,alpha_ndcg,ndcg_ia,s_recall,c_kl,c_hell,c_hell_target
a_value,,,,,,,,,,,,
greedy_0,0.224,0.670,0.282,0.425,0.563,0.550,0.890,0.518,0.383,0.797,0.364,0.360
greedy_01,0.224,0.670,0.282,0.425,0.563,0.550,0.890,0.518,0.383,0.796,0.363,0.360
greedy_02,0.224,0.670,0.282,0.426,0.564,0.549,0.890,0.519,0.383,0.793,0.363,0.359
greedy_03,0.225,0.671,0.282,0.427,0.564,0.548,0.890,0.519,0.384,0.789,0.362,0.359
greedy_04,0.225,0.672,0.282,0.430,0.565,0.545,0.890,0.520,0.386,0.775,0.359,0.356
greedy_05,0.224,0.671,0.282,0.420,0.557,0.528,0.886,0.522,0.395,0.741,0.353,0.349
greedy_06,0.223,0.669,0.281,0.415,0.553,0.519,0.887,0.522,0.399,0.736,0.352,0.348
greedy_07,0.223,0.669,0.280,0.411,0.550,0.508,0.886,0.524,0.406,0.714,0.348,0.344
greedy_08,0.222,0.667,0.279,0.401,0.541,0.475,0.875,0.523,0.427,0.668,0.339,0.335


In [523]:
complete_results_greedy.to_csv('results_perfs/greedy/complete_greedy.csv')

In [483]:
eval_baseline_k20_global['c_hell_target'] = chl_baseline

In [525]:
eval_baseline_k20_global

,Precision,Recall,F1-score,MRR,NDCG,ILS,alpha_ndcg,ndcg_ia,s_recall,c_kl,c_hell,c_hell_target
a_value,,,,,,,,,,,,
baseline_k20,0.224,0.67,0.282,0.425,0.563,0.55,0.89,0.518,0.383,0.797,0.364,0.364


In [524]:
eval_baseline_k20_global.to_csv('results_perfs/baseline/global_baseline.csv')

# Clusters evaluation

In [193]:
clusters = pd.read_csv('cluster_users.csv', index_col=0)

In [194]:
clusters.head()

,entropy,cluster_fix
504290,0.57,4
219624,0.67,5
10420,0.70,5
511303,0.69,5
184564,0.72,6


In [134]:
def evaluation_clusters(clusters_df, results, test_set, interest, similarity_matrix, categories_list, name_parameters, k=10):
    #création de deux dataframes vides où seront stockés les résultats pour chaque cluster (global et indiv)
    global_results = pd.DataFrame()
    indiv_results = pd.DataFrame()
    #Itération sur chaque cluster
    for c in range(0, clusters_df['cluster_fix'].nunique(), 1):
        #Création de liste des users d'un cluster
        users_c = clusters_df[clusters_df['cluster_fix']==c].index.tolist()
        #Filtrage données du cluster
        results_c = results[results['user_id'].isin(users_c)]
        test_c = test_set[test_set['user_id'].isin(users_c)]
        #Calcul des performances pour ce cluster
        name_parameters_c = name_parameters+'_c'+str(c)
        cluster_global, cluster_ind = get_all_results(results_c, test_c, users_c, users_interest, similarity_matrix, categories_list, name_parameters_c, k=k)
        cluster_ind.index = [str(x)+('_c')+str(c) for x in cluster_ind.index.tolist()]
        if c == 0:
            global_results = cluster_global.copy()
            indiv_results = cluster_ind.copy()
        else:
            global_results = pd.concat([global_results, cluster_global])
            indiv_results = pd.concat([indiv_results, cluster_ind])
    return global_results, indiv_results

## Baseline

In [204]:
eval_cluster_baseline, eval_indiv_cluster_baseline = evaluation_clusters(clusters, results_baseline_k20, test_baseline_k20, users_interest, similarity_matrix, categories_list, 'baseline', k=20)

INFO - Performing evaluation on metrics chosen


Pre-processing...
OK!
Accuracy metrics...


Performing NDCG@20:  100%|██████████| 5/5 [00:00<00:00]       


OK!
ILS...


100%|██████████| 3/3 [00:00<00:00, 194.32it/s]


...

OK!
Calibration KL


100%|██████████| 207/207 [00:00<00:00, 380.27it/s]


OK!
Calibration Hellinger


100%|██████████| 207/207 [00:00<00:00, 367.36it/s]

OK!


## ADF

In [201]:
eval_cluster_a0, eval_indiv_cluster_a0 = evaluation_clusters(clusters, results_a_0_k20, test_set, users_interest, similarity_matrix, categories_list, 'a_0_k20', k=20)
eval_cluster_a01, eval_indiv_cluster_a01 = evaluation_clusters(clusters, results_a_01_k20, test_set, users_interest, similarity_matrix, categories_list, 'a_01_k20', k=20)
eval_cluster_a02, eval_indiv_cluster_a02 = evaluation_clusters(clusters, results_a_02_k20, test_set, users_interest, similarity_matrix, categories_list, 'a_02_k20', k=20)
eval_cluster_a03, eval_indiv_cluster_a03 = evaluation_clusters(clusters, results_a_03_k20, test_set, users_interest, similarity_matrix, categories_list, 'a_03_k20', k=20)
eval_cluster_a04, eval_indiv_cluster_a04 = evaluation_clusters(clusters, results_a_04_k20, test_set, users_interest, similarity_matrix, categories_list, 'a_04_k20', k=20)
eval_cluster_a05, eval_indiv_cluster_a05 = evaluation_clusters(clusters, results_a_05_k20, test_set, users_interest, similarity_matrix, categories_list, 'a_05_k20', k=20)
eval_cluster_a06, eval_indiv_cluster_a06 = evaluation_clusters(clusters, results_a_06_k20, test_set, users_interest, similarity_matrix, categories_list, 'a_06_k20', k=20)
eval_cluster_a07, eval_indiv_cluster_a07 = evaluation_clusters(clusters, results_a_07_k20, test_set, users_interest, similarity_matrix, categories_list, 'a_07_k20', k=20)
eval_cluster_a08, eval_indiv_cluster_a08 = evaluation_clusters(clusters, results_a_08_k20, test_set, users_interest, similarity_matrix, categories_list, 'a_08_k20', k=20)
eval_cluster_a09, eval_indiv_cluster_a09 = evaluation_clusters(clusters, results_a_09_k20, test_set, users_interest, similarity_matrix, categories_list, 'a_09_k20', k=20)
eval_cluster_a1, eval_indiv_cluster_a1 = evaluation_clusters(clusters, results_a_1_k20, test_set, users_interest, similarity_matrix, categories_list, 'a_1_k20', k=20)

INFO - Performing evaluation on metrics chosen


Pre-processing...
OK!
Accuracy metrics...


Performing NDCG@20:  0%|          | 0/5 [00:00<?]             C:\Users\ctreuill.BIDOUILLE\AppData\Roaming\Python\Python39\site-packages\clayrs\evaluation\metrics\ranking_metrics.py:205: RuntimeWarning: invalid value encountered in double_scalars
  return actual / ideal
Performing NDCG@20:  100%|██████████| 5/5 [00:00<00:00]


OK!
ILS...


  0%|          | 0/3 [00:00<?, ?it/s]

...

OK!
Calibration KL


100%|██████████| 207/207 [00:00<00:00, 448.32it/s]


OK!
Calibration Hellinger


100%|██████████| 207/207 [00:00<00:00, 477.96it/s]

OK!


In [231]:
# a = 0
# for i in [eval_indiv_cluster_a0, eval_indiv_cluster_a01, eval_indiv_cluster_a02, eval_indiv_cluster_a03, eval_indiv_cluster_a04, eval_indiv_cluster_a05, eval_indiv_cluster_a06, eval_indiv_cluster_a07, eval_indiv_cluster_a08, eval_indiv_cluster_a09, eval_indiv_cluster_a1]:
#     i = i.round(3)
#     path = 'results_perfs/indiv/clusters/ind_a_0'+str(a)+'.csv'
#     i.to_csv(path)
#     a = a+1

In [315]:
results_complete_c0 = pd.concat([eval_cluster_baseline.iloc[:1, :],eval_cluster_a0.iloc[:1, :],eval_cluster_a01.iloc[:1, :],eval_cluster_a02.iloc[:1, :],eval_cluster_a03.iloc[:1, :],eval_cluster_a04.iloc[:1, :],eval_cluster_a05.iloc[:1, :],eval_cluster_a06.iloc[:1, :],eval_cluster_a07.iloc[:1, :],eval_cluster_a08.iloc[:1, :],eval_cluster_a09.iloc[:1, :],eval_cluster_a1.iloc[:1, :]])
results_complete_c1 = pd.concat([eval_cluster_baseline.iloc[1:2, :],eval_cluster_a0.iloc[1:2, :],eval_cluster_a01.iloc[1:2, :],eval_cluster_a02.iloc[1:2, :],eval_cluster_a03.iloc[1:2, :],eval_cluster_a04.iloc[1:2, :],eval_cluster_a05.iloc[1:2, :],eval_cluster_a06.iloc[1:2, :],eval_cluster_a07.iloc[1:2, :],eval_cluster_a08.iloc[1:2, :],eval_cluster_a09.iloc[1:2, :],eval_cluster_a1.iloc[1:2, :]])
results_complete_c2 = pd.concat([eval_cluster_baseline.iloc[2:3, :],eval_cluster_a0.iloc[2:3, :],eval_cluster_a01.iloc[2:3, :],eval_cluster_a02.iloc[2:3, :],eval_cluster_a03.iloc[2:3, :],eval_cluster_a04.iloc[2:3, :],eval_cluster_a05.iloc[2:3, :],eval_cluster_a06.iloc[2:3, :],eval_cluster_a07.iloc[2:3, :],eval_cluster_a08.iloc[2:3, :],eval_cluster_a09.iloc[2:3, :],eval_cluster_a1.iloc[2:3, :]])
results_complete_c3 = pd.concat([eval_cluster_baseline.iloc[3:4, :],eval_cluster_a0.iloc[3:4, :],eval_cluster_a01.iloc[3:4, :],eval_cluster_a02.iloc[3:4, :],eval_cluster_a03.iloc[3:4, :],eval_cluster_a04.iloc[3:4, :],eval_cluster_a05.iloc[3:4, :],eval_cluster_a06.iloc[3:4, :],eval_cluster_a07.iloc[3:4, :],eval_cluster_a08.iloc[3:4, :],eval_cluster_a09.iloc[3:4, :],eval_cluster_a1.iloc[3:4, :]])
results_complete_c4 = pd.concat([eval_cluster_baseline.iloc[4:5, :],eval_cluster_a0.iloc[4:5, :],eval_cluster_a01.iloc[4:5, :],eval_cluster_a02.iloc[4:5, :],eval_cluster_a03.iloc[4:5, :],eval_cluster_a04.iloc[4:5, :],eval_cluster_a05.iloc[4:5, :],eval_cluster_a06.iloc[4:5, :],eval_cluster_a07.iloc[4:5, :],eval_cluster_a08.iloc[4:5, :],eval_cluster_a09.iloc[4:5, :],eval_cluster_a1.iloc[4:5, :]])
results_complete_c5 = pd.concat([eval_cluster_baseline.iloc[5:6, :],eval_cluster_a0.iloc[5:6, :],eval_cluster_a01.iloc[5:6, :],eval_cluster_a02.iloc[5:6, :],eval_cluster_a03.iloc[5:6, :],eval_cluster_a04.iloc[5:6, :],eval_cluster_a05.iloc[5:6, :],eval_cluster_a06.iloc[5:6, :],eval_cluster_a07.iloc[5:6, :],eval_cluster_a08.iloc[5:6, :],eval_cluster_a09.iloc[5:6, :],eval_cluster_a1.iloc[5:6, :]])
results_complete_c6 = pd.concat([eval_cluster_baseline.iloc[6:7, :],eval_cluster_a0.iloc[6:7, :],eval_cluster_a01.iloc[6:7, :],eval_cluster_a02.iloc[6:7, :],eval_cluster_a03.iloc[6:7, :],eval_cluster_a04.iloc[6:7, :],eval_cluster_a05.iloc[6:7, :],eval_cluster_a06.iloc[6:7, :],eval_cluster_a07.iloc[6:7, :],eval_cluster_a08.iloc[6:7, :],eval_cluster_a09.iloc[6:7, :],eval_cluster_a1.iloc[6:7, :]])
results_complete_c7 = pd.concat([eval_cluster_baseline.iloc[7:8, :],eval_cluster_a0.iloc[7:8, :],eval_cluster_a01.iloc[7:8, :],eval_cluster_a02.iloc[7:8, :],eval_cluster_a03.iloc[7:8, :],eval_cluster_a04.iloc[7:8, :],eval_cluster_a05.iloc[7:8, :],eval_cluster_a06.iloc[7:8, :],eval_cluster_a07.iloc[7:8, :],eval_cluster_a08.iloc[7:8, :],eval_cluster_a09.iloc[7:8, :],eval_cluster_a1.iloc[7:8, :]])

In [316]:
results_complete_c0.to_csv('results_perfs/clusters/cluster_0.csv')
results_complete_c1.to_csv('results_perfs/clusters/cluster_1.csv')
results_complete_c2.to_csv('results_perfs/clusters/cluster_2.csv')
results_complete_c3.to_csv('results_perfs/clusters/cluster_3.csv')
results_complete_c4.to_csv('results_perfs/clusters/cluster_4.csv')
results_complete_c5.to_csv('results_perfs/clusters/cluster_5.csv')
results_complete_c6.to_csv('results_perfs/clusters/cluster_6.csv')
results_complete_c7.to_csv('results_perfs/clusters/cluster_7.csv')

## No pers

In [309]:
eval_cluster_06_nopers, eval_indiv_cluster_06_nopers = evaluation_clusters(clusters, results_glob_a06, test_set, users_interest, similarity_matrix, categories_list, 'h_06_k20', k=20)
eval_cluster_07_nopers, eval_indiv_cluster_07_nopers = evaluation_clusters(clusters, results_glob_a07, test_set, users_interest, similarity_matrix, categories_list, 'h_07_k20', k=20)
eval_cluster_08_nopers, eval_indiv_cluster_08_nopers = evaluation_clusters(clusters, results_glob_a08, test_set, users_interest, similarity_matrix, categories_list, 'h_08_k20', k=20)
eval_cluster_09_nopers, eval_indiv_cluster_09_nopers = evaluation_clusters(clusters, results_glob_a09, test_set, users_interest, similarity_matrix, categories_list, 'h_09_k20', k=20)
eval_cluster_1_nopers, eval_indiv_cluster_1_nopers = evaluation_clusters(clusters, results_glob_a1, test_set, users_interest, similarity_matrix, categories_list, 'h_1_k20', k=20)

INFO - Performing evaluation on metrics chosen


Pre-processing...
OK!
Accuracy metrics...


Performing NDCG@20:  100%|██████████| 5/5 [00:00<00:00]       


OK!
ILS...


100%|██████████| 3/3 [00:00<00:00, 116.63it/s]


...

OK!
Calibration KL


100%|██████████| 207/207 [00:00<00:00, 355.17it/s]


OK!
Calibration Hellinger


100%|██████████| 207/207 [00:00<00:00, 345.76it/s]

OK!


In [311]:
# a = 0
# for i in [eval_indiv_cluster_a0, eval_indiv_cluster_a01, eval_indiv_cluster_a02, eval_indiv_cluster_a03, eval_indiv_cluster_a04, eval_indiv_cluster_a05, eval_indiv_cluster_a06, eval_indiv_cluster_a07, eval_indiv_cluster_a08, eval_indiv_cluster_a09, eval_indiv_cluster_a1]:
#     i = i.round(3)
#     path = 'results_perfs/indiv/clusters/ind_a_0'+str(a)+'.csv'
#     i.to_csv(path)
#     a = a+1

In [312]:
eval_cluster_06_nopers

,Precision,Recall,F1-score,MRR,NDCG,ILS,alpha_ndcg,ndcg_ia,s_recall,c_kl,c_hell
a_value,,,,,,,,,,,
h_06_k20_c0,0.201,0.694,0.269,0.217,0.423,0.340,0.525,0.615,0.692,0.491,0.409
h_06_k20_c1,0.212,0.653,0.293,0.359,0.546,0.666,0.909,0.795,0.417,0.062,0.129
h_06_k20_c2,0.274,0.738,0.347,0.391,0.555,0.566,0.793,0.711,0.429,0.068,0.122
h_06_k20_c3,0.259,0.698,0.323,0.424,0.563,0.541,0.835,0.660,0.439,0.087,0.124
h_06_k20_c4,0.243,0.676,0.303,0.419,0.561,0.493,0.844,0.604,0.454,0.119,0.134
h_06_k20_c5,0.224,0.632,0.277,0.439,0.570,0.428,0.902,0.560,0.516,0.137,0.140
h_06_k20_c6,0.214,0.586,0.261,0.413,0.562,0.379,0.906,0.519,0.603,0.142,0.143
h_06_k20_c7,0.172,0.543,0.216,0.337,0.520,0.341,0.875,0.476,0.710,0.133,0.140


In [317]:
results_complete_c0_no_pers = pd.concat([eval_cluster_06_nopers.iloc[:1, :],eval_cluster_07_nopers.iloc[:1, :],eval_cluster_08_nopers.iloc[:1, :],eval_cluster_09_nopers.iloc[:1, :],eval_cluster_1_nopers.iloc[:1, :]])
results_complete_c1_no_pers = pd.concat([eval_cluster_06_nopers.iloc[1:2, :],eval_cluster_07_nopers.iloc[1:2, :],eval_cluster_08_nopers.iloc[1:2, :],eval_cluster_09_nopers.iloc[1:2, :],eval_cluster_1_nopers.iloc[1:2, :]])
results_complete_c2_no_pers = pd.concat([eval_cluster_06_nopers.iloc[2:3, :],eval_cluster_07_nopers.iloc[2:3, :],eval_cluster_08_nopers.iloc[2:3, :],eval_cluster_09_nopers.iloc[2:3, :],eval_cluster_1_nopers.iloc[2:3, :]])  
results_complete_c3_no_pers = pd.concat([eval_cluster_06_nopers.iloc[3:4, :],eval_cluster_07_nopers.iloc[3:4, :],eval_cluster_08_nopers.iloc[3:4, :],eval_cluster_09_nopers.iloc[3:4, :],eval_cluster_1_nopers.iloc[3:4, :]])  
results_complete_c4_no_pers = pd.concat([eval_cluster_06_nopers.iloc[4:5, :],eval_cluster_07_nopers.iloc[4:5, :],eval_cluster_08_nopers.iloc[4:5, :],eval_cluster_09_nopers.iloc[4:5, :],eval_cluster_1_nopers.iloc[4:5, :]])  
results_complete_c5_no_pers = pd.concat([eval_cluster_06_nopers.iloc[5:6, :],eval_cluster_07_nopers.iloc[5:6, :],eval_cluster_08_nopers.iloc[5:6, :],eval_cluster_09_nopers.iloc[5:6, :],eval_cluster_1_nopers.iloc[5:6, :]])  
results_complete_c6_no_pers = pd.concat([eval_cluster_06_nopers.iloc[6:7, :],eval_cluster_07_nopers.iloc[6:7, :],eval_cluster_08_nopers.iloc[6:7, :],eval_cluster_09_nopers.iloc[6:7, :],eval_cluster_1_nopers.iloc[6:7, :]])  
results_complete_c7_no_pers = pd.concat([eval_cluster_06_nopers.iloc[7:8, :],eval_cluster_07_nopers.iloc[7:8, :],eval_cluster_08_nopers.iloc[7:8, :],eval_cluster_09_nopers.iloc[7:8, :],eval_cluster_1_nopers.iloc[7:8, :]])  

In [326]:
results_complete_c0_no_pers.to_csv('results_perfs/no_pers/clusters/cluster_0.csv')
results_complete_c1_no_pers.to_csv('results_perfs/no_pers/clusters/cluster_1.csv')
results_complete_c2_no_pers.to_csv('results_perfs/no_pers/clusters/cluster_2.csv')
results_complete_c3_no_pers.to_csv('results_perfs/no_pers/clusters/cluster_3.csv')
results_complete_c4_no_pers.to_csv('results_perfs/no_pers/clusters/cluster_4.csv')
results_complete_c5_no_pers.to_csv('results_perfs/no_pers/clusters/cluster_5.csv')
results_complete_c6_no_pers.to_csv('results_perfs/no_pers/clusters/cluster_6.csv')
results_complete_c7_no_pers.to_csv('results_perfs/no_pers/clusters/cluster_7.csv')